In [1]:
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 226152721070306616
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4298113024
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13606944259739188919
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1660 SUPER, pci bus id: 0000:07:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Conv2D, MaxPooling2D, Dropout, Add, BatchNormalization, Activation, Reshape
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, mean_squared_error

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

my_seed = 1027

tf.random.set_seed(my_seed)
np.random.seed(my_seed)

import librosa

import os
import ast
import csv

from tqdm import tqdm
def tqdm_close():
    for instance in tqdm._instances:
        instance.close()
import multiprocessing as mp
print('cpu 개수는:', mp.cpu_count())
from joblib import Parallel, delayed        

cpu 개수는: 8


In [7]:
### make input file list
def get_file_list(cls):
    if cls == 1:
        folder_path = 'dataset' # siren
    else:
        folder_path = 'dataset2' # others
    file_list = os.listdir(folder_path)
    return file_list

def rsp_50to20k(original_array): # shape: (4,100)
    rsp_array_list = []
    fre_array_list = []
    for array in original_array:
        rsp_array = np.round(librosa.resample(array, orig_sr=50, target_sr=5000), 3)
        rsp_array_list.append(rsp_array)
        mel_spectrogram = librosa.feature.melspectrogram(y=rsp_array, sr=5000, n_mels=64, hop_length=101, win_length=2048)
        log_mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
        mfcc = librosa.feature.mfcc(S=log_mel_spectrogram, sr=5000, n_mfcc= 36)
        fre_array_list.append(np.round(np.vstack((log_mel_spectrogram, mfcc)), 3))
    sum_array = np.sum(np.array(rsp_array_list), axis=0)
    return np.array(rsp_array_list), sum_array, np.round(rsp_array/sum_array*100, 3), np.array(fre_array_list)

def np_ast(str_list):
    return np.array(ast.literal_eval(str_list))

In [8]:
ch_rsp_list, ch_rt_list = [], []
label1_list, label2_list = [], []
fre_list = []

for cls in [0, 1]:  
    
    if cls == 1:
        folder_path = 'dataset'
    else:
        folder_path = 'dataset2'
    file_list = get_file_list(cls)  

    for i in tqdm(range(len(file_list))):
        x = float(file_list[i].split('.')[0].split('_')[0])
        y = float(file_list[i].split('.')[0].split('_')[1])

        dt = round(np.sqrt(x**2 + y**2), 1)
        theta = np.arctan2(y, x)
        deg = round(theta * (180 / np.pi), 1)
        cos_val = round(np.cos(theta), 2)
        sin_val = round(np.sin(theta), 2)

        df_to_prcs = pd.read_csv(f'{folder_path}/{file_list[i]}')

        for j in range(0,151,25):
            df_sub = df_to_prcs.iloc[j:j+100, :]
            ch_rsp, sum_rsp, ch_rt, fre = rsp_50to20k(df_sub.iloc[:, :4].T.values.astype(float))

            
            ch_rsp_list.append(ch_rsp)
            ch_rt_list.append(np.array(np.vstack((ch_rt, sum_rsp))))
                              
            fre_list.append(fre)
            
            label1_list.append(np.array([x, y, cls]))
            label2_list.append(np.array([sin_val, cos_val, dt, cls]))

tqdm_close()

ch_rsp_list_tr, ch_rt_list_tr = np.array(ch_rsp_list), np.array(ch_rt_list)
label1_list_tr, label2_list_tr = np.array(label1_list), np.array(label2_list)
fre_list_tr = np.array(fre_list)

100%|████████████████████████████████████████████████████████████████████████████████| 443/443 [01:23<00:00,  5.30it/s]


In [9]:
# input layer
input_shape_1 = (4, 10000, 1) # resampling raw data
input_shape_2 = (100, 100, 4) # frequency featuring

input_layer_1 = Input(shape=input_shape_1)
input_layer_2 = Input(shape=input_shape_2)

# 1D CONV # raw feature extract
# filter  100개로 -> log mel spectrogram + mfcc의 구성 요소 수와 동일 : 각 필터가 주파수 특성과 연관
x1 = Conv2D(100, (1, 1), activation='relu')(input_layer_1) # (4ch, 10000t, 100feature)
x1 = Conv2D(100, (1, 1), activation='relu')(x1) # (4, 10000, 100)
x1 = MaxPooling2D((1, 100))(x1) # (4, 100t, 100f)
x1 = tf.transpose(x1, perm=[0, 3, 2, 1]) # (100f, 100t, 4)

# MLnet
x1 = Conv2D(32, (4, 4), (2, 2), activation='relu')(x1) # (49, 49, 32)
x1 = Conv2D(32, (4, 4), (2, 2), activation='relu')(x1) # (23, 23, 32)
x1 = MaxPooling2D((2, 2), (2, 2))(x1) #(11, 11, 32)
x1 = Flatten()(x1) #(3872)

# MLnet
x2 = Conv2D(32, (4, 4), (2, 2), activation='relu')(input_layer_2) # (49, 49, 32)
x2 = Conv2D(32, (4, 4), (2, 2), activation='relu')(x2) # (23, 23, 32)
x2 = MaxPooling2D((2, 2), (2, 2))(x2) #(11, 11, 32)
x2 = Flatten()(x2) #(3872)

x3 = Add()([x1, x2])
x3 = Dense(512, activation='relu')(x3)
x3 = Dense(64, activation='relu')(x3)
output_layer = Dense(3)(x3)

model = Model(inputs=[input_layer_1, input_layer_2], outputs=output_layer)


initial_learning_rate, min_learning_rate  = 0.001, 0.000000001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,  # 초기 학습률
    decay_steps=10000,      # 학습률을 감소시킬 스텝 수
    decay_rate=0.9,         # 학습률을 줄일 비율
    staircase=False
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

losses = ['mean_squared_error', 'mean_squared_error', tf.losses.BinaryCrossentropy(), ]
loss_weights =  [1, 1, 0.1]

model.compile(optimizer=optimizer, loss=losses, loss_weights=loss_weights)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 4, 10000, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 4, 10000, 10  200         ['input_1[0][0]']                
                                0)                                                                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 4, 10000, 10  10100       ['conv2d[0][0]']                 
                                0)                                                            

In [10]:
# 데이터를 준비하고 분할
# x_train_1 = ch_rsp_list_tr
# x_train_2 = fre_list_tr.reshape((5285,100,100,4))
# y_train = label1_list_tr  # 타겟 데이터

x_train_1, x_val_1, x_train_2, x_val_2, y_train, y_val = train_test_split(np.array(ch_rsp_list), np.array(fre_list).reshape((5285,100,100,4)), np.array(label1_list), test_size=0.2, random_state=my_seed)

# 모델 학습
batch_size = 4
epochs = 10

# Define the ModelCheckpoint callback
checkpoint = ModelCheckpoint(
    'siren_model2.h5',
    monitor='val_loss',  # Check based on validation loss|
    verbose=1,
    save_best_only=True,  # Save only the best model
    mode='min'  # Save the model when the validation loss is minimized
)

# 입력 데이터 목록을 사용하여 모델 학습
model.fit([x_train_1, x_train_2], y_train, validation_data=([x_val_1, x_val_2], y_val), batch_size=batch_size, epochs=epochs, callbacks=[checkpoint])

Epoch 1/10
1057/1057 [==============================] - 34s 23ms/step - loss: 1862.2543 - val_loss: 1458.2552
Epoch 2/10
1057/1057 [==============================] - 21s 20ms/step - loss: 1341.8911 - val_loss: 1301.5817
Epoch 3/10
1057/1057 [==============================] - 21s 20ms/step - loss: 1252.0693 - val_loss: 1216.1724
Epoch 4/10
1057/1057 [==============================] - 21s 20ms/step - loss: 1169.2623 - val_loss: 1594.8536
Epoch 5/10
1057/1057 [==============================] - 21s 20ms/step - loss: 1069.3506 - val_loss: 1206.6265
Epoch 6/10
1057/1057 [==============================] - 21s 20ms/step - loss: 970.4026 - val_loss: 1115.6079
Epoch 7/10
1057/1057 [==============================] - 22s 20ms/step - loss: 882.2443 - val_loss: 1541.0095
Epoch 8/10
1057/1057 [==============================] - 21s 20ms/step - loss: 794.3475 - val_loss: 1122.5950
Epoch 9/10
1057/1057 [==============================] - 21s 20ms/step - loss: 674.8948 - val_loss: 1081.6191
Epoch 10/10
10

In [11]:
y_hat = model.predict([x_val_1, x_val_2])

# 실제 y 값과 예측값 비교
rmse = np.sqrt(np.mean((y_hat-y_val)**2, axis=0, keepdims=True))
print()
print(f"x_rmse: {round(rmse[0][0],1)}, y_rmse: {round(rmse[0][1],1)}")
print()

for i in range(10):
    print("Sample", i)
    print("Actual:", y_val[i])
    print("Predicted:", y_hat[i])

34/34 [==============================] - 3s 50ms/step

x_rmse: 50.2, y_rmse: 31.2

Sample 0
Actual: [  0. -35.   0.]
Predicted: [-36.314972  -60.10395     0.9579242]
Sample 1
Actual: [115.  71.   0.]
Predicted: [28.95109    20.738724   -0.09037858]
Sample 2
Actual: [75. 45.  0.]
Predicted: [ 8.0241837e+01  4.5434059e+01 -4.7613323e-02]
Sample 3
Actual: [-75.   5.   1.]
Predicted: [-45.448067    11.1883545    0.85691184]
Sample 4
Actual: [65. 15.  0.]
Predicted: [-15.57131    40.491215   -0.2916252]
Sample 5
Actual: [-30.  20.   1.]
Predicted: [-16.568935   15.679655    0.5092759]
Sample 6
Actual: [  0. -85.   1.]
Predicted: [-99.45501   28.129692   1.150605]
Sample 7
Actual: [-140.   30.    1.]
Predicted: [-93.74896    17.834906    1.1419313]
Sample 8
Actual: [115.  15.   1.]
Predicted: [75.24126   21.002716   1.0367808]
Sample 9
Actual: [80. 20.  1.]
Predicted: [51.32849    17.8663      0.40869194]
